<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L7_code/Lecture7_demo1_extract_features_pyradiomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting features from a segmented lesion with PyRadiomics

This demo shows how to extract intensity and shape based features form segmented masses in mammography using the [PyRadiomics](https://www.radiomics.io/pyradiomics.html) pythin package.
A number of mammography images and mass segmentatation masks are available in the [shared folder on Drive](https://drive.google.com/drive/folders/1YqK7ZkM-P2IrqfD7Pj-SCmjz-GWd_1-Y )

in IMAGES/Mammography_masses/


# Reading data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -q /content/gdrive/My\ Drive/cmepda_medphys_dataset/IMAGES/Mammography_masses/small_sample_Im_segmented_ref.zip -d /content/

dataset_path = "/content/small_sample_Im_segmented_ref"

In [ ]:
!ls /content/small_sample_Im_segmented_ref

# Overview of the dataset.
We use [Pillow](https://pypi.org/project/Pillow/), a Python Imaging Library which adds image processing capabilities to the Python interpreter

In [ ]:
import os
import PIL

In [ ]:
PIL.Image.open(os.path.join(dataset_path, "0008p1_3_1_2_resized.pgm"))

In [ ]:
PIL.Image.open(os.path.join(dataset_path, "0008p1_3_1_2_mass_mask.pgm"))

# Install PyRadiomics

In [ ]:
!pip install pyradiomics

# Use PyRadiomics for feature extraction

In [ ]:
import os
import numpy as np
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics

The feature extractor handles preprocessing, and then calls the needed featureclasses to calculate the features.

In [ ]:
featureClasses = getFeatureClasses()

In [ ]:
featureClasses

We have to initialize the feature extractor and we can customize extraction settings 

In [ ]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

Input images: by default 'original' (no filtered) images are enabled. Optionally different image types can be enabled, e.g. 

In [ ]:
extractor.enableImageTypeByName('Wavelet')

To check the enabled input images

In [ ]:
for imageType in extractor.enabledImagetypes.keys():
    print('\t' + imageType)

To disable all feature classes

In [ ]:
extractor.disableAllFeatures()

To enable all features in firstorder

In [ ]:
extractor.enableFeatureClassByName('firstorder')

Alternatively: only enable 'Mean' and 'Skewness' features in firstorder

In [ ]:
extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])

In [ ]:
extractor.enabledFeatures


We can get the docstrings of the active features

In [ ]:
print('Active features:')
for cls, features in six.iteritems(extractor.enabledFeatures):
  if len(features) == 0:
    features = [f for f, deprecated in six.iteritems(featureClasses[cls].getFeatureNames()) if not deprecated]
  for f in features:
    print(f)
    print(getattr(featureClasses[cls], 'get%sFeatureValue' % f).__doc__)

# Reading the images in memory 

PyRadiomics accept [SimpleITK](https://simpleitk.org/) objects in input

In [ ]:
import SimpleITK as sitk

Out images are ".pgm" files. This format is not supported by SimpleITK 

In [ ]:
os.path.join(dataset_path, "0008p1_3_1_2_resized.pgm")

We read the images with Pillow and store them in NumPy array, than we convert them in a SimpleITK object

In [ ]:
def read_pgm_as_sitk(image_path):
  """ Read a pgm image as sitk image """
  np_array = np.asarray(PIL.Image.open(image_path))
  sitk_image = sitk.GetImageFromArray(np_array)
  return sitk_image

In [ ]:
im_1 = read_pgm_as_sitk(os.path.join(dataset_path, "0008p1_3_1_2_resized.pgm"))
im_1_mask = read_pgm_as_sitk(os.path.join(dataset_path, "0008p1_3_1_2_mass_mask.pgm"))

In [ ]:
type(im_1)

# Calculating the values of active features

In [ ]:
print('Calculating features')
featureDict = extractor.execute(im_1,im_1_mask,label=255)
type(featureDict)

Features are stored in a dictionary

In [ ]:
featureDict

# Compute the features for the whole dataset and store them in a file 

We will compute the features for the whole dataset, and we will add to the dictionary the case IDs




In [ ]:
import glob
import csv

In [ ]:
images_fnames = glob.glob(os.path.join(dataset_path,'*_resized.pgm'))

In [ ]:
extracted_data = []
for image_fname in images_fnames:
  mask_fname = image_fname.replace('resized', 'mass_mask')
  image = read_pgm_as_sitk(image_fname)
  mask = read_pgm_as_sitk(mask_fname)
  featureVector = extractor.execute(image, mask, label=255)
  featureVector['image_ID'] = os.path.basename(image_fname)
  featureVector['mask_ID'] = os.path.basename(mask_fname)
  extracted_data.append(featureVector)


In [ ]:
extracted_data

The extracted data is a list of dictionary

In [ ]:
type(extracted_data)

We will identify the variables we will store in the .csv file. 

In [ ]:
list(extracted_data[0].keys())

We can either store them all: 

In [ ]:
csv_columns =  list(extracted_data[0].keys())

or we can select a number of interesting features:

In [ ]:
selected_features_name = [x for x in list(extracted_data[0].keys()) if 'firstorder' in x]
csv_columns = [ 'image_ID', 'mask_ID' ] + selected_features_name
csv_columns 


We will store the selected features for further analysis in a .csv file

In [ ]:
csv_file = "extracted_features.csv"

with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns, extrasaction='ignore' )
    writer.writeheader()
    for data in extracted_data:
        writer.writerow(data)

In [ ]:
!ls

In [ ]:
!cat extracted_features.csv

We can copy the output file on our gdrive folder, as the /content/ forlder content will be reset after the session

In [ ]:
!cp extracted_features.csv /content/gdrive/My\ Drive/.